# Obtaining Data sets

In this notebook, the codes for obtaining datasets and the proper dataframes (for the features) is shown.

### Feature Engineering


The features that we will be using are the following:

Creating a data set where we include:
- Historical Returns of all stocks included in the index, and the index.
- Rows: time
- Columns: Stocks / index


Another dataset including:
- Spreads of the stocks and index.
- Rows: time
- Columns: Stocks / index

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime

# Day to Day Trading


In the day to day historical OHLC information we are using this in order to obtain a portfolio that takes into account the spread (cost) and find an optimal portfolio that reduces the tracking loss between the the portfolio and the index we are referring to.

To begin, the Dow Jones Industrial Average will be used as an index.

We can obtain the historical daily prices day to day for 10 years using Yahoo Finance API. With this we will obtain the days included.

**Dow Jones Index Data:**

In [ ]:
# Day to day of Dow Jones ^DJI
DJI = yf.Ticker('^DJI')

print(DJI.info)

DJI_hist = DJI.history(period = '10y')

# Using a simple high and low spread
DJI_hist['HLSpread'] = DJI_hist['High']-DJI_hist['Low']

DJI_hist
# type(DJI_hist['Date'][0])
DJI_hist.index
# DJI_hist['Datecol'] = DJI_hist.index

# Dividends and Stocks Splits are not needed at all for an Index.
DJI_hist = DJI_hist.drop('Dividends', axis=1)
DJI_hist = DJI_hist.drop('Stock Splits', axis=1)


DJI_hist
DJI_hist.to_csv('DJI.csv')

# myhist = pd.read_csv('DJI.csv', index_col = [0])
# myhist

Obtaining the stock data from yahoo finance and getting the return, log return, and the High Low Spread.

$$

TE = sd(r_{index} - r_{portfolio})\\

\;\\

r = \text{return} = \dfrac{Price_{t} - Price_{t-1}}{Price_{t-1}} \;=\; \dfrac{Price_{t}}{Price_{t-1}} - 1\\

\;\\

\boxed{\text{log returns} \;=\;    ln\left(\dfrac{Price_{t}}{Price_{t-1}} \right)}\\

\;\\


r_{portfolio} = \sum_{i=1}^{num stocks} r_{i, \;t} = \sum_{i=1}^{num stocks}  \left(\dfrac{Price_{t} - Price_{t-1}}{Price_{t}}\right) _{i. \;t}
$$

In [ ]:
# Function for any sort of STOCK or index.
def obtain_stockdata(ticker:str, period = '10y'):

    try:

        # Retrieving from yfinance API the historical data.
        mystock         = yf.Ticker(ticker)
        historical_data = mystock.history(period = '10y')

        # OBTAINING: Spread ... high - low.
        # Using a simple high and low spread since the bid and ask historical data is not allowed.
        historical_data['HLSpread'] = historical_data['High']-historical_data['Low']

        # Dividends and Stocks Splits are not needed at all for an index.
        historical_data = historical_data.drop('Dividends', axis=1)
        historical_data = historical_data.drop('Stock Splits', axis=1)

        # OBTAINING: Return ... the return of the stock in percentage.
        
        close_data = pd.Series(historical_data['Close']) # must be a series data type

        # calculating the percentage in return (*100). [(x_f - x_i)/x_i] * 100.
        # Removed the *100 since we want to compare tko log return
        historical_data['Return'] = close_data.pct_change()#*100

        # log(1+x) ~ x for x->inf.
        historical_data['LogReturn'] = np.log(close_data/close_data.shift(1))
        # removing the first row since we dont want an NA value.
        historical_data.drop(index=historical_data.index[0], axis=0, inplace=True)

        return historical_data
    
    except:
        print("Incorrect Stock symbol. Choose a correct one!")

# Obtaining the Dow Jones stock data
DJI = obtain_stockdata('^DJI')
DJI.head()


# Saving the data to a csv
DJI.to_csv('DJI_features.csv')

Saving the basic Dow Jones index csv file.

In [ ]:
DJI = pd.read_csv('DJI_features.csv', index_col = [0])
DJI.head()

Since we have a function that automatically calculates the return, spread and log returns, we can use this to find these features for each of the stocks included in the index.

In [ ]:
# All names of the stocks included in the DOW Jones Industrial Index.
DJ_stock_list = ['^DJI','AMZN', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD',
                 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM','MCD','MMM', 'MRK', 'MSFT',
                 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WMT', 'DIS', 'DOW']

# storing all the returns of all stocks into one dataframe.
return_df = pd.DataFrame(index=DJI.index)

# storing all the spreads of all stocks into one dataframe.
spread_df = pd.DataFrame(index=DJI.index)

logreturn_df = pd.DataFrame(index=DJI.index)

for each in DJ_stock_list:

    temp_stock_data = obtain_stockdata(each)
    # stock_str = str(each)
    return_df[str(each)] = temp_stock_data['Return']
    logreturn_df[str(each)] = temp_stock_data['LogReturn']
    spread_df[str(each)] = temp_stock_data['HLSpread']
    print('{} done!'.format(each))

Because information on DOW is only since 2019, it is better not to include it since we may want more data.

In [ ]:
# print(return_df.isna())
# DOW began in 2019, we can remove it and not include it for the day to day since we would reduce the amount of observations...

try:
    return_df = return_df.drop('DOW', axis=1)
    logreturn_df = logreturn_df.drop('DOW', axis=1)
    spread_df = spread_df.drop('DOW', axis=1)
    print('Removed DOW column correctly.')
except:
    print('Already Removed or did not exist.')

Saving to a csv file:

In [ ]:
# Saving each dataframe to a csv file

# HighLow Spread ....
spread_df.to_csv('spread_df.csv')
# Return ....
return_df.to_csv('return_df.csv')
# Log Return ....
logreturn_df.to_csv('logreturn_df.csv')

Reading from a csv file:
- We need to also include the column used as the index (csv not saved with indices but rather a new column was created as the index.)

In [ ]:
print("\nSpreads --------------")
# index_col = [0] 
spread_df = pd.read_csv('spread_df.csv', index_col = [0])
spread_df.index = pd.to_datetime(spread_df.index)

print(spread_df.describe())



print("\nReturns --------------")
# index_col = [0] 
return_df = pd.read_csv('return_df.csv', index_col = [0])
return_df.index = pd.to_datetime(return_df.index)
print(return_df.describe())



print("\nLogReturns --------------")
# index_col = [0]
logreturn_df = pd.read_csv('logreturn_df.csv', index_col = [0])
logreturn_df.index = pd.to_datetime(logreturn_df.index)
print(logreturn_df.describe())

### Intraday Trading data (Alpha Vantage)

Intraday data can be obtained using Alpha Vantage. Note how we can obtain a JSON or csv file. If we want to change the parameters of the function then we need to do so in the url. For example, extended_hours = False is to be included in the url we obtain the data from. Also note how the data is in string format. We need to change it to a float in order to properly use it.

In [ ]:
def get_data(symbol:str, interval = "5min",extended_hours = False):

    # Only use data from trading hours, not extended
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    
    if type(symbol) == str:
        try:
            # Extended hours = false
            url_str = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol="+str(symbol)+"&interval="+interval+"&extended_hours=False"+"&apikey=M5CNIDBZ8YEVD8AN"
        except:
            print("Not a correct stock symbol")
    else:
        print("Symbol is not a string, converting to a string. Make sure it is a correct stock name!")
        symbol = str(symbol)
        url_str = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol="+str(symbol)+"&interval="+interval+"&extended_hours=False"+"&apikey=M5CNIDBZ8YEVD8AN"

    
    url = str(url_str)#'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBEX&interval=5min&apikey=M5CNIDBZ8YEVD8AN'
    r = requests.get(url)
    data = r.json()
    return data


data = get_data("SPY")
try:
    print(data['Meta Data'])
    print(data.keys())
    print(data['Time Series (5min)'])

except:
    print("Not a correct stock symbol.")


# values of prices are taken from a JSON file and are strings! CHANGE TO FLOATS.
times      = [each for each in data['Time Series (5min)'].keys()]
stock_data = [each for each in data['Time Series (5min)'].values()]

## - note how they are in string format!
open       = [float(each['1. open']) for each in stock_data] 
high       = [float(each['2. high']) for each in stock_data]
low        = [float(each['3. low']) for each in stock_data]
close      = [float(each['4. close']) for each in stock_data]
volume     = [float(each['5. volume']) for each in stock_data]

timestamps = pd.to_datetime(times)
timestamps


stock = pd.DataFrame({
"Date":pd.to_datetime(times),
"open":open, 
"high":high,  
"low":low,      
"close":close,    
"volume":volume})
stock.head()
stock['spread'] = stock['high']-stock['low']

plt.plot(stock['Date'], stock['close'])
plt.show

stock.head()